In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [ ]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

In [ ]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("../data/locations.txt", encoding='UTF-8')

data = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

#text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n",], chunk_size=1000, chunk_overlap=100)
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=500, chunk_overlap=50)

all_splits = text_splitter.split_documents(data)

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embed_model)

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
from llm_integration import HotelTool
tools = [HotelTool]
llm_with_tools = llm.bind_tools(tools)



In [ ]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)

In [ ]:
title = "Travel Assistant"	
examples = [
    "I want to travel to Japan for 8 days from 2024-06-15 to 2024-06-22 with a budget of 5000 dollars, I will be travelling alone.",
    "I want to travel to Mumbai for 8 days from 2024-06-15 to 2024-06-22 with a budget of 5000 dollars, I will be travelling with a partner."
]

In [ ]:
import gradio as gr

gr.ChatInterface(
    fn = llm_cycle,
    title=title,
    examples=examples
).launch()